In [1]:
# Source paper: https://pubs.aip.org/aip/jrse/article/13/2/025301/926842

In [2]:
import csv
from cvxpy import *
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [3]:
import data_processing as dp

pv_directory = 'PV Generation Data'
load_directory = 'Building Load Data'
wind_directory = 'Wind Generation Data'

#dp.generate_json_from_pv_data(pv_directory) ALREADY GENERATED!
#dp.generate_json_from_bldg_data(load_directory) ALREADY GENERATED!
#dp.generate_json_from_wind_data(wind_directory) ALREADY GENERATED!

Reading: Wind Generation Data/WATERLWF.csv
Reading: Wind Generation Data/HALLWF1.csv
Reading: Wind Generation Data/HALLWF2.csv
Reading: Wind Generation Data/LKBONNY2.csv
Reading: Wind Generation Data/LKBONNY3.csv
Reading: Wind Generation Data/LKBONNY1.csv
Reading: Wind Generation Data/WAUBRAWF.csv
Reading: Wind Generation Data/SNOWTWN1.csv
Reading: Wind Generation Data/GUNNING1.csv
Reading: Wind Generation Data/CATHROCK.csv
Reading: Wind Generation Data/WOODLWN1.csv
Reading: Wind Generation Data/CLEMGPWF.csv
Reading: Wind Generation Data/WOOLNTH1.csv
Reading: Wind Generation Data/NBHWF1.csv
Reading: Wind Generation Data/OAKLAND1.csv
Reading: Wind Generation Data/YAMBUKWF.csv
Reading: Wind Generation Data/MTMILLAR.csv
Reading: Wind Generation Data/BLUFF1.csv
Reading: Wind Generation Data/WPWF.csv
Reading: Wind Generation Data/STARHLWF.csv
Reading: Wind Generation Data/CULLRGWF.csv
Reading: Wind Generation Data/CAPTL_WF.csv


In [ ]:
# Import data for IEEE 39-bus Test Feeder

# Define transformer resistance and reactance (not provided by test feeder)
xformer_r = 0.0001
xformer_x = 0.0015

# Adjacency matrix; assumes power can flow in two directions on all lines
A_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='A matrix')
A_df = A_df.iloc[:,1:] # remove the column labels
A    = A_df.values     # convert to a numpy array

# Resistance matrix
r_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='r matrix')
r_df = r_df.iloc[:,1:] # remove the column labels
r_df.replace('??', xformer_r, inplace=True)
r    = r_df.values     # convert to a numpy array

# Reactance matrix
x_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='x matrix')
x_df = x_df.iloc[:,1:] # remove the column labels
x_df.replace('??', xformer_x, inplace=True)
x    = x_df.values     # convert to a numpy array

In [ ]:
# Import PV generation data (takes about 5 minutes to read-in)
directory = 'PV Generation Data'
pv_data = pd.DataFrame()

# Iterate through all files in the directory
for file in os.listdir(directory):
    
    if file.endswith(".csv"):                                      # only iterate over csv files
        filepath = os.path.join(directory, file)                   # Get the full file path
        filename = os.path.splitext(os.path.basename(filepath))[0] # use filename as column name
        
        print("Reading:", filepath)
        data_in = pd.read_csv(filepath, parse_dates=['DateTime'])

        # Normalize the data
        data_in['RealPower'] /= data_in['RealPower'].max()

        # Replace negative values with zeros
        data_in.loc[data_in['RealPower'] < 0, 'RealPower'] = 0
        
        print(data_in.shape)
        plt.figure(figsize=(8, 4))
        plt.plot(data_in['DateTime'], data_in['RealPower'])
        plt.xlabel('Date')
        plt.ylabel('PV Generation (kW)')
        plt.title(filename)
        plt.show()

        pv_data[filename] = data_in['RealPower']  # only extract real power, not datetime


In [ ]:
# Get date column
for file in os.listdir(directory):
    if file.endswith(".csv"):                    # only iterate over csv files
        filepath = os.path.join(directory, file) # Get the full file path
        data_in = pd.read_csv(filepath, parse_dates=['DateTime'])
        pv_data['date'] = data_in.iloc[:, 0]     # only extract datetime
        break # only need to extract date column from one of the files

In [ ]:
# Try to visualize the data (just try one PV generator set to start)

# Full time period
for column in pv_data.columns:
    plt.figure(figsize=(100, 10))
    plt.scatter(pv_data.index, pv_data[column])
    plt.xlabel('Date')
    plt.ylabel('PV Generation (kW)')
    plt.title(column)
    plt.show()
    break

# Approximately one day
for column in pv_data.columns:
    plt.figure(figsize=(8, 4))
    plt.scatter(pv_data.index[0:96], pv_data[column][0:96])
    plt.xlabel('Data point index')
    plt.ylabel('PV Generation (kW)')
    plt.title(column)
    plt.show()
    break

# Plot multiple days for just one PV generator (plot every 15 days, otherwise overwhelming)
num_data_points = pv_data.shape[0]
day_length = 96 # data point every 15 minutes --> 4 points per hour, 24 hours per day
num_days = int(np.floor(num_data_points/day_length))

for column in pv_data.columns:
    
    plt.figure(figsize=(8, 4)) 
    for day in range(0, num_days, 15):
        start = day_length * day
        end = start + day_length       
        plt.plot(range(0, day_length), pv_data[column][start:end], label=f'day {day + 1}')
    
    plt.xlabel('Data point index')
    plt.ylabel('PV Generation (kW)')
    plt.title(column)
    plt.legend(loc='upper center', bbox_to_anchor=(1.3, 0.85), ncol=3)
    plt.show()
    
    break


In [ ]:
# DECISION: 
# - Use 427 days worth of data starting from March 2015, because that is what is common across all of the PV generators
# - For each building, collect the daily profiles into their own dataframe, with columns as days and rows as times
# - These profiles can be deterministically or randomly selected as the generation profile for a given day in the optimal control problem

# Create a dictionary, keyed by building name, with daily data over the 427 days
pv_dict = {}
for column in pv_data.columns:
    daily_data = np.zeros([num_days, day_length])

    for day in range(num_days):
        start = day_length * day
        end = start + day_length       
        daily_data[day, :] = pv_data[column][start:end]  

    pv_dict[column] = daily_data.tolist() # use LoL as numpy arrays are not json serializable

import json
with open('PV Generation Data/pv_data.json', 'w') as json_file:
    json.dump(pv_dict, json_file)


In [ ]:
# Check that the json data was written correctly
with open('PV Generation Data/pv_data.json', 'r') as json_file:
    pv_dict_r = json.load(json_file)

    for key in pv_dict_r.keys(): # test on first key only
        daily_data = np.array(pv_dict_r[key])
        plt.plot(range(daily_data.shape[1]), daily_data[422, :]) # e.g. for day 422
        break


In [ ]:
# NOW LET'S DO ALL THE SAME THINGS FOR THE BUILDING LOAD DATA

In [ ]:
# Import building load data (takes about 5 minutes to read-in)
directory = 'Building Load Data'
real_load = pd.DataFrame()
reactive_load = pd.DataFrame()

# Iterate through all files in the directory
for file in os.listdir(directory):
    
    if file.endswith(".csv"):                                      # only iterate over csv files
        filepath = os.path.join(directory, file)                   # Get the full file path
        filename = os.path.splitext(os.path.basename(filepath))[0] # use filename as column name
        
        print("Reading:", filepath)
        data_in = pd.read_csv(filepath, parse_dates=['DateTime'])
        print(data_in.shape)

        fig, ax = plt.subplots(2, figsize=(8, 4)) 
        fig.suptitle(filename, fontsize=16) 

        ax[0].plot(data_in.iloc[:,0], data_in.iloc[:,1])
        ax[0].set_xlabel('Date')
        ax[0].set_ylabel('Real Load (kW)')

        ax[1].plot(data_in.iloc[:,0], data_in.iloc[:,2])
        ax[1].set_xlabel('Date')
        ax[1].set_ylabel('Reactive Load (kW)')
        plt.tight_layout()
        plt.show()

        real_load[filename] = data_in.iloc[:, 1] # real power in column B of excel sheet
        reactive_load[filename] = data_in.iloc[:, 2] # real power in column C of excel sheet

In [ ]:
# Get date column
for file in os.listdir(directory):
    if file.endswith(".csv"):                    # only iterate over csv files
        filepath = os.path.join(directory, file) # Get the full file path
        data_in = pd.read_csv(filepath, parse_dates=['DateTime'])
        real_load['date'] = data_in.iloc[:, 0]     # only extract datetime
        reactive_load['date'] = data_in.iloc[:, 0]     # only extract datetime
        break # only need to extract date column from one of the files

In [ ]:
# Try to visualize the data (just try one building to start)

# Full time period
for column in real_load.columns:

    fig, ax = plt.subplots(2, figsize=(100, 10)) 
    fig.suptitle(column, fontsize=16) 

    ax[0].scatter(real_load.index, real_load[column])
    ax[0].set_xlabel('Date')
    ax[0].set_ylabel('Real Load (kW)')

    ax[1].scatter(reactive_load.index, reactive_load[column])
    ax[1].set_xlabel('Date')
    ax[1].set_ylabel('Reactive Load (kW)')
    plt.tight_layout()
    plt.show()

    break

# Approximately one day
for column in real_load.columns:
    
    fig, ax = plt.subplots(2, figsize=(8, 4)) 
    fig.suptitle(column, fontsize=16) 

    ax[0].scatter(range(96), real_load[column][0:96])
    

    ax[1].scatter(range(96), reactive_load[column][0:96])
    ax[1].set_xlabel('Date')
    ax[1].set_ylabel('Reactive Load (kW)')
    plt.tight_layout()
    plt.show()

    break

# Plot multiple days for just one PV generator (plot every 15 days, otherwise overwhelming)
for column in real_load.columns:
    
    fig, ax = plt.subplots(2, figsize=(8, 4)) 
    fig.suptitle(column, fontsize=16)

    for day in range(0, num_days, 15):
        start = day_length * day
        end = start + day_length       
        
        ax[0].plot(range(0, day_length), real_load[column][start:end], label=f'day {day + 1}')
        ax[0].set_xlabel('Date')
        ax[0].set_ylabel('Real Load (kW)')

        ax[1].plot(range(0, day_length), reactive_load[column][start:end])
        ax[1].set_xlabel('Date')
        ax[1].set_ylabel('Reactive Load (kW)')
    
    plt.show()
    
    break

In [ ]:
# Create a dictionary, keyed by building name, with daily data over the 427 days
real_dict = {}
reactive_dict = {}

for column in real_load.columns:
    real_data     = np.zeros([num_days, day_length])
    reactive_data = np.zeros([num_days, day_length])

    for day in range(num_days):
        start = day_length * day
        end = start + day_length       
        real_data[day, :]     = real_load[column][start:end]  
        reactive_data[day, :] = reactive_load[column][start:end]  

    real_dict[column] = real_data.tolist() # use LoL as numpy arrays are not json serializable
    reactive_dict[column] = reactive_data.tolist()

with open('Building Load Data/real_power_data.json', 'w') as json_file:
    json.dump(real_dict, json_file)

with open('Building Load Data/reactive_power_data.json', 'w') as json_file:
    json.dump(reactive_dict, json_file)

In [ ]:
# Check that the json data was written correctly
with open('Building Load Data/real_power_data.json', 'r') as json_file:
    real_dict_r = json.load(json_file)

    for key in real_dict_r.keys(): # test on first key only
        real_data = np.array(real_dict_r[key])
        plt.plot(range(real_data.shape[1]), real_data[422, :]) # e.g. for day 422
        break

In [ ]:
# Open the original CSV file for reading
with open('Wind Generation Data/wind_data_5min.csv', mode='r') as original_file:
    csv_reader = csv.reader(original_file)
    
    # Open a new CSV file for writing
    with open('Wind Generation Data/wind_data_15min.csv', mode='w', newline='') as new_file:
        csv_writer = csv.writer(new_file)
        
        # Read and write the header row
        header_row = next(csv_reader)
        csv_writer.writerow(header_row)
        
        # Use a counter to keep track of the rows
        row_counter = 0
        
        # Iterate over the remaining rows in the original CSV file
        for row in csv_reader:
            # Write every third row to the new CSV file, starting after the header row
            if row_counter % 3 == 0:
                csv_writer.writerow(row)
            
            # Increment the row counter
            row_counter += 1

print("New CSV file created successfully.")

In [ ]:
wind_data_15_min = pd.read_csv('Wind Generation Data/wind_data_15min.csv')

print(wind_data_15_min.columns)
directory = 'Wind Generation Data'

date = wind_data_15_min['Date']
time = wind_data_15_min['Time']

for col in wind_data_15_min.columns:
    if (col == 'Date') or (col == 'Time'):
        continue
    turbine_df = wind_data_15_min[['Date', 'Time', col]]
    filename = col + '.csv'
    filepath = os.path.join(directory, filename)
    turbine_df.to_csv(filepath, index=False)